In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # Can select any from the below:
    # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"
    # "unsloth/Qwen2.5-14B",  "unsloth/Qwen2.5-32B",  "unsloth/Qwen2.5-72B",
    # And also all Instruct versions and Math. Coding verisons!
    model_name = "unsloth/Qwen2.5-14B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ltnga/ITDSIU21079/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: OpenAI failed to import - ignoring for now.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.2: Fast Qwen2 patching. Transformers:4.47.0.
   \\   /|    GPU: NVIDIA A100-SXM4-80GB. Max memory: 79.252 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [08:01<00:00, 240.74s/it]


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.2 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


In [21]:
from datasets import Dataset
import json
def prepare_dataset(json_file: str) -> Dataset:
    """
    Load and prepare the dataset from a JSON file.
    
    Args:
        json_file: Path to the JSON file containing the dataset
        
    Returns:
        A Hugging Face Dataset
    """
    # Load the data
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    processed_data = []
    
    # Process each example
    for item in data:
        # Format: Question + reasoning + final answer
        prompt = f"Hỏi: {item['question']}\n\n"
        response = f"Suy luận ban đầu: {item['initial_reasoning']}\n\n"
        
        # Add iterations
        for i, iteration in enumerate(item['iterations']):
            response += f"Bước {i+1}:\n"
            response += f"Truy vấn: {iteration['query']}\n"
            response += "Tài liệu tham khảo:\n"
            for doc in iteration['retrieved_documents']:
                response += f"- {doc}\n"
            response += f"Suy luận: {iteration['reasoning']}\n\n"
        
        # Add final answer
        response += f"Câu trả lời cuối cùng: {item['final_answer']}"
        
        # Create a formatted example
        processed_data.append({
            "prompt": prompt,
            "response": response,
            "combined": prompt + response
        })
    
    # Convert to HF Dataset
    return Dataset.from_list(processed_data)

In [24]:
from typing import Dict, List
def tokenize_dataset(examples: Dict[str, List], tokenizer, max_length: int = 512) -> Dict[str, List]:
    """
    Tokenize the dataset examples.
    
    Args:
        examples: Dictionary containing the examples
        tokenizer: The tokenizer to use
        max_length: Maximum sequence length
        
    Returns:
        Dictionary with tokenized examples
    """
    # Tokenize inputs
    model_inputs = tokenizer(
        examples["combined"],
        max_length=max_length,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    
    # Create the labels (same as input_ids for causal LM)
    model_inputs["labels"] = model_inputs["input_ids"].clone()
    
    return model_inputs

In [25]:
dataset = prepare_dataset("/home/ltnga/ITDSIU21079/auto_training.json")
print(len(dataset))

154


In [26]:
max_length = 2048  # Adjust based on your requirements
tokenized_train = dataset.map(
    lambda examples: tokenize_dataset(examples, tokenizer, max_length),
    batched=True,
    remove_columns=dataset.column_names
)

Map: 100%|██████████| 154/154 [00:00<00:00, 1011.76 examples/s]


In [27]:
from trl import SFTTrainer
from transformers import TrainingArguments, Trainer
from unsloth import is_bfloat16_supported

trainer = Trainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tokenized_train,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "qwen-vietnam-traffic",
        report_to = "none", # Use this for WandB etc
    ),
)

/tmp/ipykernel_245006/742708929.py:5: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 154 | Num Epochs = 4
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 68,812,800


Step,Training Loss
1,64.516200
2,65.627600
3,62.541100
4,51.864700
5,39.112600
6,35.719800
7,35.047600
8,31.607000
9,31.238500
10,31.600700
